# 프로세스 파악
- 이상치 까지 전처리 후,  같은caseID의 같은taskID 에서 start, complete가 둘다 존재하는 taskID에서 둘중 한 행을 삭제 후 시간 정렬과 caseID그룹화를 통해 프로세스 파악을 한다

In [56]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv(r'C:\Users\82106\Desktop\과제테스트\repair2.csv')

In [57]:
df['taskID'].unique()

array(['InformClientSurvey', 'ArrangeSurvey', 'ReadyInformClient',
       'Survey', 'TicketReady', 'MakeTicket', 'InternRepair',
       'FirstContact', 'RepairReady', 'SendTicketToFinAdmin',
       'InformClientWrongPlace', 'ImmediateRepair', 'ExternRepair', nan],
      dtype=object)

In [58]:
#'taskID', 'originator', 'eventtype' 결측치 존재 행 삭제
df = df.dropna(subset=['taskID', 'originator', 'eventtype','date','time'])

# 이외의 결측치는 NaN으로 채우기
df[['contact', 'RepairType', 'objectKey', 'RepairInternally', 'EstimatedRepairTime', 'RepairCode', 'RepairOK']]= df[['contact', 'RepairType', 'objectKey', 'RepairInternally', 'EstimatedRepairTime', 'RepairCode', 'RepairOK']].fillna(pd.NA)

# date와 time 컬럼을 합쳐 datetime 컬럼 생성
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')

# caseID와 taskID를 기준으로 피벗하여 start와 complete 이벤트를 하나의 행으로 결합
pivot = df.pivot_table(index=['caseID', 'taskID'],
                       columns='eventtype',
                       values='datetime',
                       aggfunc='min')

# start와 complete 모두 존재하는 행에 대해 시간 차이 계산 
mask_both = pivot['start'].notna() & pivot['complete'].notna()
pivot.loc[mask_both, 'time_diff'] = pivot.loc[mask_both, 'complete'] - pivot.loc[mask_both, 'start']

# 시간 차이가 너무 큰 행에서 complete 또는 start 중 하나만 삭제
threshold = pd.Timedelta(days=100)

# start가 남아야 하는 경우 complete만 삭제
pivot.loc[mask_both & (pivot['time_diff'] > threshold), 'complete'] = pd.NaT

# complete가 남아야 하는 경우 start만 삭제
pivot.loc[mask_both & (pivot['time_diff'] > threshold), 'start'] = pd.NaT

# 피벗 테이블을 다시 원본 형식으로 변환
pivot = pivot.stack().reset_index().rename(columns={0: 'datetime'})

# 데이터 타입 문제 해결을 위해 datetime 컬럼 타입 일관성 유지
pivot['datetime'] = pd.to_datetime(pivot['datetime'], errors='coerce')
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# 다시 원본 데이터와 병합하여 반영
df = df.merge(pivot, on=['caseID', 'taskID', 'eventtype', 'datetime'], how='right')

# 결과 확인을 위해 정렬
df = df.sort_values(by=['caseID', 'datetime'])

In [ ]:
# 같은 caseID와 taskID 그룹으로 묶어 처리
def drop_duplicate_eventtypes(group):
    # group 내 eventtype의 집합
    event_set = set(group['eventtype'])
    # start와 complete가 모두 존재하면 그룹에서 첫 번째 행만 반환
    if 'start' in event_set and 'complete' in event_set:
        return group.iloc[[0]]
    # 그렇지 않으면 그룹 전체 반환
    return group

# 그룹별로 적용하여 필터링
df = df.groupby(['caseID', 'taskID'], group_keys=False).apply(drop_duplicate_eventtypes)

C:\Users\82106\AppData\Local\Temp\ipykernel_23536\3082783574.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['caseID', 'taskID'], group_keys=False).apply(drop_duplicate_eventtypes)


In [60]:
# caseID별로 그룹화 후 시간 정렬
df = df.sort_values(by=['caseID', 'datetime'])

# caseID별로 프로세스 파악을 위한 그룹핑
grouped = df.groupby('caseID').agg({'taskID': list, 'datetime': list, 'eventtype': list})

In [61]:
grouped[['taskID','eventtype']]

,taskID,eventtype
caseID,,
1,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, start, complete, start, star..."
2,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, start, complete, start, star..."
3,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, start, complete, start, star..."
4,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, complete, complete, start, s..."
5,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, start, complete, start, star..."
...,...,...
996,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, start, complete, start, star..."
997,"[FirstContact, MakeTicket, ArrangeSurvey, Info...","[complete, start, start, complete, start, star..."
998,"[FirstContact, ArrangeSurvey, InformClientSurv...","[complete, start, complete, start, start, comp..."


# 프로세스 유형별 정리
| Process Type | Count | Flow | 설명 |
|-------------|-------|------|------|
| **Type 1**  | 77    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → InternRepair → RepairReady → SendTicketToFinAdmin → ReadyInformClient | 최초 접수 후 티켓 생성 및 조사 일정 조정이 한 번씩 진행되며, 조사 후 내부 수리, 수리 완료, 고객 안내, 재무 관리자에게 티켓 전송 순으로 진행됨. |
| **Type 2**  | 69    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → ImmediateRepair → RepairReady → ReadyInformClient → SendTicketToFinAdmin | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 진행 후 즉시 수리, 수리 완료, 고객 안내, 재무 관리자에게 티켓 전송 순으로 진행됨. |
| **Type 3**  | 67    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → InternRepair → RepairReady → ReadyInformClient → SendTicketToFinAdmin | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 진행 후 내부 수리, 수리 완료, 고객 안내, 재무 관리자에게 티켓 전송 순으로 진행됨. |
| **Type 4**  | 63    | FirstContact | 최초 접수 후 추가 작업 없이 종료됨 (미완료 요청 혹은 처리되지 않은 사례). |
| **Type 5**  | 48    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → ImmediateRepair → RepairReady → SendTicketToFinAdmin → ReadyInformClient | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 후 즉시 수리, 수리 완료, 재무 관리자에게 티켓 전송 및 고객 안내 순으로 진행됨. |
| **Type 6**  | 21    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → InternRepair → RepairReady → ReadyInformClient | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 후 내부 수리, 수리 완료 및 고객 안내 순으로 진행됨. |
| **Type 7**  | 17    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → InternRepair → RepairReady → SendTicketToFinAdmin → ReadyInformClient | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 후 내부 수리, 수리 완료, 재무 관리자에게 티켓 전송 및 고객 안내 순으로 진행됨. |
| **Type 8**  | 17    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → ExternRepair → RepairReady → SendTicketToFinAdmin → ReadyInformClient | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 후 외부 수리, 수리 완료, 재무 관리자에게 티켓 전송 및 고객 안내 순으로 진행됨. |
| **Type 9**  | 15    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → InternRepair → RepairReady → ReadyInformClient → SendTicketToFinAdmin | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사와 내부 수리 진행, 수리 완료, 고객 안내, 재무 관리자에게 티켓 전송 순으로 진행됨. |
| **Type 10** | 14    | FirstContact → MakeTicket → ArrangeSurvey → InformClientSurvey → Survey → RepairReady → ReadyInformClient → SendTicketToFinAdmin | 최초 접수 후 티켓 생성과 조사 일정 조정, 조사 후 바로 수리 완료, 고객 안내, 재무 관리자에게 티켓 전송 순으로 진행됨. |


### 🔍 분석 요약

1. **일반적인 프로세스 흐름**  
   - 대부분의 프로세스는 **FirstContact(최초 접수)** 후 **MakeTicket(티켓 생성)** 및 **ArrangeSurvey(조사 일정 조정)** 단계를 거침.
   - 이후 **InformClientSurvey(조사 일정 안내) → Survey(조사 진행)** 과정이 진행됨.

2. **수리 유형 분류**  
   - **Survey(조사 진행)** 이후 다음 중 하나가 수행됨:
     - **InternRepair(내부 수리)**
     - **ImmediateRepair(즉시 수리)**
     - **ExternRepair(외부 수리)**  
   - 수리가 완료되면 **RepairReady(수리 준비 완료) → ReadyInformClient(고객에게 준비 완료 안내) → SendTicketToFinAdmin(재무 관리자에게 티켓 전송)** 과정을 거침.

3. **미완료된 요청 가능성**  
   - **Process Type 4**는 **FirstContact(최초 접수)** 이후 추가적인 작업이 진행되지 않음.
   - 이는 미완료된 요청이거나 초기 단계에서 중단된 사례일 가능성이 있음.



##### 병목현상 분석 : 다음 프로세스까지 시간계산 (기간차이 이상치는 제거하고 진행)

In [70]:
# 병목 현상 분석: 동일 taskID 내에서 평균 수행 시간이 긴 경우 찾기

# 기간 차이가 많이 나는 행 삭제
threshold_days = pd.Timedelta(days=5000)
df['next_datetime'] = df.groupby('caseID')['datetime'].shift(-1)
df['time_diff'] = df['next_datetime'] - df['datetime']

# 기간 이상 차이가 나는 행(이상치) 삭제
df = df[df['time_diff'] <= threshold_days]

# taskID -> 다음 taskID로 수행 시간 분석
df['next_taskID'] = df.groupby('caseID')['taskID'].shift(-1)
task_transition_time = df.groupby(['taskID', 'next_taskID'])['time_diff'].mean().reset_index()

# 일정 시간 이상 걸리는 작업 흐름을 병목 현상으로 분류 (평균 수행 시간 1일 이상)
bottleneck_threshold = pd.Timedelta(days=1)
bottleneck_transitions = task_transition_time[task_transition_time['time_diff'] > bottleneck_threshold]

# 병목 현상 분석 결과를 긴 순서대로 정렬
bottleneck_transitions_sorted= bottleneck_transitions.sort_values(by='time_diff', ascending=False).reset_index(drop=True)

In [71]:
bottleneck_transitions_sorted

,taskID,next_taskID,time_diff
0,MakeTicket,Survey,7 days 21:37:30
1,ArrangeSurvey,InternRepair,6 days 18:52:00
2,InformClientSurvey,ExternRepair,3 days 14:51:15
3,InformClientSurvey,Survey,3 days 05:46:51.269841269
4,InformClientSurvey,InternRepair,3 days 05:09:43.846153846
5,InformClientSurvey,ImmediateRepair,3 days 02:31:22.500000
6,ArrangeSurvey,Survey,3 days 00:22:20
7,Survey,InternRepair,1 days 07:48:56.099071207


### 🔍 병목현상 주요 패턴  

##### 1️. MakeTicket → Survey (7.9일)  
- 고객 요청이 등록된 후, 조사(Survey)가 진행되기까지 상당한 대기 시간이 발생.  
- **주요 원인:** 일정 조정 문제, 조사 인력 부족  

##### 2️. ArrangeSurvey → InternRepair (6.8일)  
- 조사 완료 후, 내부 수리 진행까지 일정이 지연됨.  
- **주요 원인:** 부품 준비, 기술자 배정  

##### 3️. InformClientSurvey 관련 지연 (3~3.5일)  
- InformClientSurvey(고객 연락 후 안내) 이후 Survey, InternRepair, ImmediateRepair로 가는 과정에서 3일 이상 지연됨.  
- **주요 원인:** 고객 응답 대기, 일정 충돌  

##### 4️. Survey → InternRepair (1.3일)  
- 조사 완료 후 내부 수리로 진행되기까지 1일 이상 지연됨.  
- **주요 원인:** 조사 결과 검토, 내부 인력 배정  


#####  반복 작업 분석 : 동일 caseID 내에서 같은 taskID가 여러 번 수행되는 경우

- eventtype 에서 같은 caseID의 같은 taskID에 'start' 와 'complete' 모두 존재할경우 'start' 만 남겨두기.

In [64]:
# 모든 caseID에서 동일한 taskID가 2번 이상 수행된 경우 찾기
task_repeat_count = df.groupby(['caseID', 'taskID']).size().reset_index(name='count')

# 2번 이상 수행된 taskID만 필터링
task_repeat_count = task_repeat_count[task_repeat_count['count'] > 1]

# 전체 프로세스에서 taskID별 반복 횟수 집계
task_repeat_summary = task_repeat_count.groupby('taskID')['count'].sum().reset_index()

# 같은 caseID 내에서 각 taskID가 평균 몇 번 반복되었는지 계산
average_repeats = task_repeat_count.groupby('taskID')['count'].mean().reset_index()
average_repeats = average_repeats.rename(columns={'count': 'avg_repeats_per_caseID'})

# 반복 횟수와 평균 반복 횟수를 병합
task_repeat_summary = task_repeat_summary.merge(average_repeats, on='taskID')

# 반복 작업이 많이 발생한 순으로 정렬
task_repeat_summary = task_repeat_summary.sort_values(by='count', ascending=False).reset_index(drop=True)

In [65]:
task_repeat_summary

,taskID,count,avg_repeats_per_caseID


반복 작업은 존재하지 않음

### 케이스 소요 시간에 영향을 미치는 데이터 요소 파악 
🔹 **병목 현상이 있는 Task 전환 지점** → MakeTicket → Survey 과정에서 대기 시간이 많음.  
🔹 **RepairType(수리 방식)** → 즉시 수리보다 내부/외부 수리가 시간이 오래 걸림.  
🔹 **Originator(담당자)의 업무 속도 차이** → 특정 담당자가 처리할 때 시간이 길어지는 경우 존재.  
🔹 **최초 접수 후 첫 업무까지의 대기 시간** → FirstContact 이후 첫 번째 업무까지 시간이 길면 전체 케이스가 길어짐.  

##### 🚀 개선 방향  

### MakeTicket → Survey 병목 현상 해결  
- 조사 일정을 더 빠르게 배정할 수 있도록 프로세스 최적화.  

### InternRepair / ExternRepair 지연 해결  
- 내부 수리 프로세스 개선 및 외부 수리 업체와의 협업 강화.  

### 업무 담당자별 업무 속도 분석 후 교육 진행  
- 속도가 느린 담당자에게 효율적인 작업 방식 교육 제공.  

### 최초 접수 후 지연 시간 단축  
- FirstContact 후 24시간 내에 첫 작업을 시작하도록 **SLA(서비스 수준 계약)** 설정.  
